In [2]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
training_data = [
    ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
    ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
    ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
    ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
]

wordidx = {}

for sentence, tags in training_data:
    for word in sentence:
        if word not in wordidx:
            wordidx[word] = len(wordidx)

tagidx = {"DET":0, "NN":1, "V":2}
print(wordidx)

{'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


In [4]:
def prepare_sequence(seq, to_idx):
    idxs = [to_idx[s] for s in seq]
    idxs = np.array(idxs)
    return torch.from_numpy(idxs).long()

In [5]:
test = "the dog ate the book".lower().split()
test = prepare_sequence(test, wordidx)
print(test)

tensor([0, 8, 2, 0, 7])


In [28]:
class Tagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(Tagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim), torch.zeros(1, 1, self.hidden_dim))
        
    def forward(self, sentence):
        embs = self.word_embedding(sentence)
        lstm_out, self.hidden = self.lstm(embs.view(len(sentence), 1, -1), self.hidden)
        output = self.hidden2tag(lstm_out.view(len(sentence), -1))
        score = F.log_softmax(output, dim=1)
        return score
    



In [40]:
embedding_dim = 6
hidden_dim = 6

model = Tagger(embedding_dim, hidden_dim, len(wordidx), len(tagidx))
loss_func = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)



In [42]:
epochs = 1000

for epoch in range(epochs):
    epoch_loss = 0.0
    for sentence, tags in training_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        
        sentence_in = prepare_sequence(sentence, wordidx)
        targets = prepare_sequence(tags, tagidx)
        
        tag_scores = model(sentence_in)
        
        loss = loss_func(tag_scores, targets.long())
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    if(epoch%20 == 19):
        print(f'epoch:{epoch+1}, loss:{epoch_loss/len(training_data)}')
            
            
    
   

epoch:20, loss:0.01086307200603187
epoch:40, loss:0.009799887891858816
epoch:60, loss:0.008915995713323355
epoch:80, loss:0.00817066291347146
epoch:100, loss:0.007534525590017438
epoch:120, loss:0.006985788117162883
epoch:140, loss:0.006508013000711799
epoch:160, loss:0.006088558118790388
epoch:180, loss:0.005717641790397465
epoch:200, loss:0.0053874533623456955
epoch:220, loss:0.005091767758131027
epoch:240, loss:0.004825560201425105
epoch:260, loss:0.004584735550452024
epoch:280, loss:0.004365860659163445
epoch:300, loss:0.004166160535532981
epoch:320, loss:0.003983290342148393
epoch:340, loss:0.0038151669432409108
epoch:360, loss:0.0036601516185328364
epoch:380, loss:0.003516792959999293
epoch:400, loss:0.0033838643576018512
epoch:420, loss:0.00326027802657336
epoch:440, loss:0.0031450792448595166
epoch:460, loss:0.003037491755094379
epoch:480, loss:0.0029367448878474534
epoch:500, loss:0.0028422390460036695
epoch:520, loss:0.0027534711407497525
epoch:540, loss:0.002669883775524795


In [43]:
print(wordidx)

{'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


In [54]:
test_sentence = "the cheese loves the elephant".lower().split()
inputs = prepare_sequence(test_sentence, wordidx)
scores = model(inputs)
_, predicted_tags = torch.max(scores, 1)


In [55]:
print(predicted_tags)


tensor([0, 1, 2, 0, 1])


In [58]:
print(tagidx)

{'DET': 0, 'NN': 1, 'V': 2}
